# Building Sense Embeddings

The sense embeddings of a certain sense is calculated by averaging the context embeddings of all context in which certain sense exists. There exists several different methods for combining words embeddings to form context embeddings. Our starting poing is applying plain average (bag of word). 

Reference: Iaacobaci et al, Embeddings for Word Sense Disambiguation: An Evaluation Study
http://aclweb.org/anthology/P/P16/P16-1085.pdf

In [2]:
# Import neccesary libraries
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import semcor
import numpy as np
import collections
import os
import pickle
import dill

In [3]:
#Load an example embeddings
embedding_dict = pickle.load(open('glove_50d_50kvoc.pk','rb'))
example_sentence = semcor.sents()[0]

In [4]:
example_chunk = semcor.tagged_sents(tag='sem')[0]

In [5]:
example_sentence_list = semcor.tagged_sents(tag='sem')[:10]

In [6]:
# Build a function to combine word to form context embedding:
def getContextEmb(sentence,center,window_size,embedding_dict,emb_size):
    # Input introductions
    # sentence: an array of tokens of untagged sentence. 
    # center: position of the center word
    # window_size: size of context window
    # embedding_Dict: embedding dictionary used to calculate context
    ################################################################
    start_pos = max([0,center-window_size])
    end_pos = min([len(sentence),(center+window_size)+1])
    context_tokens = sentence[start_pos:end_pos]
    output_embedding = np.zeros(emb_size)
    for word in context_tokens:
        try:
            output_embedding+=embedding_dict[word]
        except:
            output_embedding+=np.random.uniform(1,-1,emb_size)
    return output_embedding

Trying to create a method to form a dictionary of sense embeddings.

In [9]:
def buildSemEmb(tagged_sents,emb_size,embedding_dict,context_builder = getContextEmb):
    output_dict = collections.defaultdict(lambda: np.zeros(emb_size))
    count_dict = collections.defaultdict(lambda: 0)
    for sentence in tagged_sents:
        #print(sentence)
        for idx,chunk in enumerate(sentence):
            if(type(chunk))==list:
                continue
            else:
                #Use try except handling since some of the label is broken
                try:
                    sense_index = chunk.label().synset().name()
                except:
                    continue
                context_emb = context_builder(sentence,idx,3,embedding_dict,emb_size)
                output_dict[sense_index]+=context_emb
                count_dict[sense_index]+=1
    # Averaging
    for key in output_dict.keys():
        output_dict[key] /= count_dict[key]
    return output_dict

Now we build a sense embedding dictionary for prediction. Notice that the ouput dictionary of buildSemEmb() is a collection.defaultdict() with default value being the uniform random vector. Hence it returns a uniform random vector when some sense does not exists.

In [10]:
#Build sense dictionary for semcor corpus
semcor_senseEmb = buildSemEmb(semcor.tagged_sents(tag='sem'),50,embedding_dict)

In [11]:
semcor_senseEmb['commitment.n.03']

array([-2.06353012, -1.12259322,  0.28076305,  0.09776931,  1.85757217,
       -2.19357166, -3.87155826, -0.31175339, -0.22573833, -2.41525063,
       -0.75352122, -1.35417784, -2.01442702,  0.33819836,  1.42471455,
       -0.78893443,  0.74265565,  0.90160286,  0.87213678,  0.56625424,
       -0.48795326, -0.74756141, -0.12013804, -1.57954475,  0.01679202,
        1.98118495, -2.51956198,  0.23062062, -3.43711529, -0.40733263,
        2.14332114,  0.96309226, -1.16615147,  0.45657349, -2.37953211,
        2.11408698,  0.75594413,  0.51281912,  1.28792596,  5.58092432,
        0.77154844, -0.74235638, -0.65335784, -0.88883975,  0.52656539,
       -1.33210767, -2.76492188, -1.66266053,  1.39222615, -2.04672858])

## Expriment: bag of word comparison with sense embeddings

Using a trained embeddings and the sense embeddings that we derived by averaging the context. We can build a classifier that directly compare the bag of words (the average embeddings of the entire sentence) with sense embeddings and output the sense with highest cosine similarity.

In [109]:
example_chunk[11].label().synset()

Synset('produce.v.04')

In [112]:
example_word = 'primary'
example_context = getContextEmb(center=15,emb_size=50,embedding_dict=embedding_dict,sentence=example_sentence,window_size=2)

In [113]:
from scipy.spatial.distance import cosine

In [114]:
choices = [synset.name() for synset in wn.synsets('produced')]

In [115]:
decision_chart = [(choice,cosine(example_context,semcor_senseEmb[choice])) for choice in choices]

In [116]:
decision_chart

[('produce.v.01', 0.80118819294311239),
 ('produce.v.02', 1.0297011816849664),
 ('produce.v.03', 1.133994408503096),
 ('produce.v.04', 1.0277081404913799),
 ('grow.v.07', 1.1651610929005445),
 ('produce.v.06', 0.93871288285740107),
 ('grow.v.08', 1.2429046816452995)]

In [117]:
#kind of fail